# Computing and Visualizing EMRI Waveform Derivatives

This tutorial demonstrates how to compute numerical derivatives of EMRI waveforms using the StableEMRIDerivative class.

We'll show how to:
- Set up waveform parameters
- Compute numerical derivatives with respect to specific parameters
- Visualize the derivative waveforms
- Understand the derivative structure

## Import Required Packages

Import the necessary packages for EMRI waveform generation and derivative computation.

In [ ]:
# Import relevant EMRI packages
from few.waveform import (
    GenerateEMRIWaveform,
    FastKerrEccentricEquatorialFlux,
)
from few.utils.constants import YRSID_SI

# Import StableEMRIDerivative
from stableemrifisher.fisher.stablederivative import StableEMRIDerivative
from stableemrifisher.fisher.derivatives import derivative

import matplotlib.pyplot as plt
import numpy as np

print(f"Year in seconds: {YRSID_SI:.2f}")

## Define Waveform Parameters

Set up the physical parameters for our EMRI system.

In [ ]:
# Physical parameters for the EMRI system
wave_params = {
    "m1": 1e6,        # Primary mass (solar masses)
    "m2": 1e1,        # Secondary mass (solar masses)
    "a": 0.9,         # Dimensionless spin parameter
    "p0": 10,         # Initial semi-latus rectum
    "e0": 0.4,        # Initial eccentricity
    "xI0": 1.0,       # Initial inclination (cos(inclination))
    "dist": 1.0,      # Distance to source (Gpc)
    "qS": 0.2,        # Sky location polar angle
    "phiS": 0.8,      # Sky location azimuthal angle
    "qK": 1.6,        # Kerr spin polar angle
    "phiK": 1.5,      # Kerr spin azimuthal angle
    "Phi_phi0": 2.0,  # Initial phase in phi
    "Phi_theta0": 0.0, # Initial phase in theta
    "Phi_r0": 3.0,    # Initial phase in r
}

print("EMRI system parameters:")
for key, value in wave_params.items():
    print(f"  {key}: {value}")

## Configure Waveform Generation

Set up the waveform class and generator with high accuracy settings.

In [ ]:
# Waveform class configuration
waveform_class_kwargs = {
    "inspiral_kwargs": {
        "err": 1e-11,  # High accuracy integration
    },
    "mode_selector_kwargs": {
        "mode_selection_threshold": 1e-5  # Include important modes
    },
}

# Waveform generator configuration
waveform_generator = GenerateEMRIWaveform
waveform_generator_kwargs = {
    "return_list": False,
    "frame": "detector"  # Generate detector frame waveforms
}

print("Waveform generation configured for high accuracy derivative computation.")

## Initialize Stable Derivative Computer

Create the derivative computation object that will handle numerical differentiation.

In [ ]:
# Initialize the StableEMRIDerivative object
EMRI_deriv = StableEMRIDerivative(
    FastKerrEccentricEquatorialFlux,
    waveform_generator=GenerateEMRIWaveform,
    waveform_generator_kwargs=waveform_generator_kwargs,
)

print("StableEMRIDerivative initialized for robust numerical differentiation.")

## Compute Derivative with Respect to Primary Mass

Calculate the derivative of the waveform with respect to the primary mass (m1) using a 4th order central finite difference scheme.

In [ ]:
# Time domain parameters
T = 0.01   # Observation time (years)
dt = 10.0  # Time step (seconds)
kwargs = {"T": T, "dt": dt}

print(f"Computing derivative with respect to m1...")
print(f"Observation time: {T} years")
print(f"Time step: {dt} seconds")
print(f"Total samples: {int(T * YRSID_SI / dt)}")

# Compute stable derivative
compute_stable_deriv = EMRI_deriv(
    parameters=wave_params,
    param_to_vary="m1",      # Parameter to differentiate with respect to
    delta=1e-1,              # Finite difference step size
    order=4,                 # 4th order finite difference
    kind="central",          # Central difference scheme
    **kwargs,
)

print(f"Derivative computed successfully!")
print(f"Derivative shape: {compute_stable_deriv.shape}")
print(f"Derivative dtype: {compute_stable_deriv.dtype}")

## Visualize the Derivative

Plot the real part of the derivative to see how the waveform changes with respect to the primary mass.

In [ ]:
# Create time array
t = np.arange(0, T * YRSID_SI, dt)

# Create the plot
plt.figure(figsize=(12, 8))

# Plot the real part of the derivative
plt.subplot(2, 1, 1)
plt.plot(t, compute_stable_deriv.real, 'b-', linewidth=1.5, label="∂h/∂m₁ (real part)")
plt.xlabel("Time (s)", fontsize=14)
plt.ylabel("Derivative Amplitude", fontsize=14)
plt.title("EMRI Waveform Derivative with respect to Primary Mass (m₁)", fontsize=16)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=12)

# Plot the imaginary part as well
plt.subplot(2, 1, 2)
plt.plot(t, compute_stable_deriv.imag, 'r-', linewidth=1.5, label="∂h/∂m₁ (imaginary part)")
plt.xlabel("Time (s)", fontsize=14)
plt.ylabel("Derivative Amplitude", fontsize=14)
plt.title("Imaginary Part of the Derivative", fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=12)

plt.tight_layout()
plt.show()

# Print some statistics
print(f"\nDerivative Statistics:")
print(f"Real part - Max: {np.max(np.abs(compute_stable_deriv.real)):.2e}")
print(f"Imaginary part - Max: {np.max(np.abs(compute_stable_deriv.imag)):.2e}")
print(f"Total amplitude - Max: {np.max(np.abs(compute_stable_deriv)):.2e}")

## Compare with Different Parameters

Let's compute derivatives with respect to different parameters to see how they compare.

In [ ]:
# Compute derivatives for different parameters
params_to_test = ["m1", "m2", "a"]
derivatives = {}
step_sizes = {"m1": 1e-1, "m2": 1e-3, "a": 1e-4}  # Different step sizes for different parameters

print("Computing derivatives for multiple parameters...")

for param in params_to_test:
    print(f"  Computing ∂h/∂{param}...")
    derivatives[param] = EMRI_deriv(
        parameters=wave_params,
        param_to_vary=param,
        delta=step_sizes[param],
        order=4,
        kind="central",
        **kwargs,
    )

print("All derivatives computed!")

## Compare Derivative Amplitudes

Plot the derivatives for different parameters to see their relative magnitudes and behavior.

In [ ]:
# Create comparison plot
plt.figure(figsize=(15, 10))

colors = ['blue', 'red', 'green']
labels = ['∂h/∂m₁', '∂h/∂m₂', '∂h/∂a']

for i, (param, color, label) in enumerate(zip(params_to_test, colors, labels)):
    plt.subplot(3, 1, i+1)
    plt.plot(t, np.abs(derivatives[param]), color=color, linewidth=1.5, label=f"{label} (magnitude)")
    plt.xlabel("Time (s)", fontsize=12)
    plt.ylabel("Derivative Magnitude", fontsize=12)
    plt.title(f"Derivative magnitude with respect to {param}", fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.legend(fontsize=11)
    plt.yscale('log')  # Log scale to see the full range

plt.tight_layout()
plt.show()

# Print maximum amplitudes for comparison
print("\nMaximum derivative amplitudes:")
for param in params_to_test:
    max_amp = np.max(np.abs(derivatives[param]))
    print(f"  ∂h/∂{param}: {max_amp:.2e}")

## Summary

This tutorial demonstrated how to compute numerical derivatives of EMRI waveforms using StableEMRIDerivative:

1. **Setup**: Configure waveform generation with high accuracy
2. **Derivative Computation**: Use stable finite difference methods to compute ∂h/∂θ
3. **Visualization**: Plot derivatives to understand their structure and behavior
4. **Comparison**: Compare derivatives for different parameters

Key observations:
- Different parameters have vastly different derivative magnitudes
- The derivative structure reflects the underlying waveform evolution
- Proper step size selection is crucial for accurate derivatives

These derivatives are the building blocks for Fisher information matrix computation, enabling rapid parameter estimation for EMRI sources.